In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.chdir('/home/export/soheuny/SRFinder/soheun')

In [4]:
# 1. Make directory for moving files
from tst_info import TSTInfo
from training_info import TrainingInfoV2
import os
import uuid
import shutil

def copy_tst_files(source_tst_hashes: list):
    def tst_tinfo_linker(tstinfo_hash: str):
        tstinfo = TSTInfo.load(tstinfo_hash)
        base_fvt_hash = tstinfo.base_fvt_tinfo_hash
        base_fvt_checkpoint = f"{base_fvt_hash}_best.ckpt"
        CR_fvt_hash = tstinfo.CR_fvt_tinfo_hash
        CR_fvt_checkpoint = f"{CR_fvt_hash}_best.ckpt"
        
        return base_fvt_hash, base_fvt_checkpoint, CR_fvt_hash, CR_fvt_checkpoint

    # source_files = tst_tinfo_linker(hashes[0])
    random_dir_name = str(uuid.uuid4())
    data_source_dir = TSTInfo.SAVE_DIR / ".."
    checkpoints_source_dir = TSTInfo.SAVE_DIR / ".." / ".." / "checkpoints"

    tinfo_target_dir = data_source_dir / random_dir_name / "training_info"
    tstinfo_target_dir = data_source_dir / random_dir_name / "tst_info"
    checkpoints_target_dir = data_source_dir / random_dir_name / "checkpoints"

    os.makedirs(data_source_dir / random_dir_name, exist_ok=True)
    os.makedirs(tinfo_target_dir, exist_ok=True)
    os.makedirs(tstinfo_target_dir, exist_ok=True)
    os.makedirs(checkpoints_target_dir, exist_ok=True)

    for tst_hash in source_tst_hashes:
        base_fvt_hash, base_fvt_checkpoint, CR_fvt_hash, CR_fvt_checkpoint = tst_tinfo_linker(tst_hash)
        
        tst_source = TSTInfo.SAVE_DIR / tst_hash
        base_fvt_source = TrainingInfoV2.SAVE_DIR / base_fvt_hash
        base_fvt_checkpoint_source = checkpoints_source_dir / base_fvt_checkpoint
        CR_fvt_source = TrainingInfoV2.SAVE_DIR / CR_fvt_hash
        CR_fvt_checkpoint_source = checkpoints_source_dir / CR_fvt_checkpoint
        
        tst_target = tstinfo_target_dir / tst_hash
        base_fvt_target = tinfo_target_dir / base_fvt_hash
        base_fvt_checkpoint_target = checkpoints_target_dir / base_fvt_checkpoint
        CR_fvt_target = tinfo_target_dir / CR_fvt_hash
        CR_fvt_checkpoint_target = checkpoints_target_dir / CR_fvt_checkpoint
        
        source_targets = [
            (tst_source, tst_target),
            (base_fvt_source, base_fvt_target),
            (base_fvt_checkpoint_source, base_fvt_checkpoint_target),
            (CR_fvt_source, CR_fvt_target),
            (CR_fvt_checkpoint_source, CR_fvt_checkpoint_target),
        ]
        
        for source, target in source_targets:
            shutil.copy2(source, target)

In [5]:
import yaml


config_filename = "configs/counting_test_more_CR_train_data.yml"

config = yaml.safe_load(open(config_filename, "r"))
experiment_name = config["experiment_name"]
n_3b = config["n_3b"]
ratio_4b = config["ratio_4b"]

hashes, hparams = TSTInfo.find({
    "experiment_name": experiment_name,
}, return_hparams=True)


1523it [00:02, 759.21it/s]


In [7]:
copy_tst_files(hashes)